In [1]:
import numpy as np
import pandas as pd

## Exploration

In [3]:
# Figure out: how to merge all csv's together, then use aggregation to group subsets
prices = pd.read_csv('sell_prices.csv')
prices.head(2)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58


prices df:
- store_id ties to sales store_id
- item_id ties to sales item_id
- wm_yr_wk ties to dates wm_yr_wk

In [5]:
dates = pd.read_csv('calendar.csv')
dates.head(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0


In [7]:
dates.tail(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0
1968,2016-06-19,11621,Sunday,2,6,2016,d_1969,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0


In [9]:
dates.event_name_1.value_counts()

MemorialDay            6
PresidentsDay          6
Purim End              6
SuperBowl              6
Mother's day           6
LentStart              6
NBAFinalsStart         6
StPatricksDay          6
NBAFinalsEnd           6
Pesach End             6
Ramadan starts         6
ValentinesDay          6
LentWeek2              6
ColumbusDay            5
Easter                 5
Christmas              5
IndependenceDay        5
Cinco De Mayo          5
LaborDay               5
Eid al-Fitr            5
Thanksgiving           5
MartinLutherKingDay    5
OrthodoxEaster         5
EidAlAdha              5
Halloween              5
VeteransDay            5
NewYear                5
OrthodoxChristmas      5
Chanukah End           5
Father's day           4
Name: event_name_1, dtype: int64

In [11]:
dates.event_type_1.value_counts()

Religious    55
National     52
Cultural     37
Sporting     18
Name: event_type_1, dtype: int64

In [10]:
dates.event_name_2.value_counts()

Father's day      2
OrthodoxEaster    1
Cinco De Mayo     1
Easter            1
Name: event_name_2, dtype: int64

In [12]:
dates.event_type_2.value_counts()

Cultural     4
Religious    1
Name: event_type_2, dtype: int64

dates df:
- wday specifies which day of week is which number: so the week starts on Saturday and ends on Friday
- the first month is Jan 2011, but the first day is Jan 29
- the last day is Jun 19 2016, d_1969 - but the sales df goes up to d_1913
    - (1969 - 1913) = 56 extra days in sales
- month column is simply the number of the month ie 01 = Jan, 06 = Jun, 12 = Dec
- year is similar. also, there is a little less than 6 years of data here!!
- event_name 1 & 2 seem like they're necessary because holidays overlap
    - to-do: make new df of just holidays as defined by WalMart, and use that for fbProphet instead of the default holiday df

In [6]:
sales = pd.read_csv('sales_train_validation.csv')
sales.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


56

## Ideas

- Prep
    1. Merge all csv's together, then use aggregation to group subsets
    2. create individual csv's for each store
    3. create individual csv's for each dept
    4. add SNAP days as Holidays in prophet
    5. make new df of just holidays as defined by WalMart, and use that for fbProphet instead of the default holiday df

- Model
    1. Use fbProphet to forecast sales for each:
        - dept in each individual store
        - dept in each state
        - individual store in each state
        - each state
        - all together
    2. Compare the models and see performance difference
        - find factors affecting the gap between predicted and actual sales
        - use those factors as modifiers to adjust the prediction generated by prophet